## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np 

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Thompson,CA,55.7435,-97.8558,33.80,93,90,14.97,light snow
1,1,Dolores,AR,-36.3132,-57.6792,66.67,88,36,7.94,scattered clouds
2,2,Albany,US,42.6001,-73.9662,59.00,22,20,14.97,few clouds
3,3,Kaitangata,NZ,-46.2817,169.8464,64.00,45,16,15.01,few clouds
4,4,Camiri,BO,-20.0500,-63.5167,80.60,69,90,4.09,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Camiri,BO,-20.0500,-63.5167,80.60,69,90,4.09,overcast clouds
6,6,Saint Anthony,US,45.0205,-93.2180,84.20,28,20,11.50,few clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,86.00,66,20,17.27,few clouds
10,10,Galesong,ID,-5.3166,119.3661,75.20,100,20,2.30,few clouds
11,11,Gao,ML,16.6362,1.6370,84.07,5,2,8.93,clear sky
15,15,Alyangula,AU,-13.8483,136.4192,81.25,53,8,2.55,clear sky
21,21,Rikitea,PF,-23.1203,-134.9692,78.39,67,46,13.78,scattered clouds
24,24,Vaini,TO,-21.2000,-175.2000,80.60,69,20,11.50,few clouds
27,27,Saint-Philippe,RE,-21.3585,55.7679,78.80,69,0,6.91,clear sky
30,30,Kapaa,US,22.0752,-159.3190,80.60,57,75,23.02,broken clouds


In [9]:
# 4a. Determine if there are any empty rows.
clean_df.count()

# no empty rows

City_ID                216
City                   216
Country                216
Lat                    216
Lng                    216
Max Temp               216
Humidity               216
Cloudiness             216
Wind Speed             216
Current Description    216
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

# NA

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Camiri,BO,80.60,overcast clouds,-20.0500,-63.5167,
6,Saint Anthony,US,84.20,few clouds,45.0205,-93.2180,
9,Puerto Ayora,EC,86.00,few clouds,-0.7393,-90.3518,
10,Galesong,ID,75.20,few clouds,-5.3166,119.3661,
11,Gao,ML,84.07,clear sky,16.6362,1.6370,
15,Alyangula,AU,81.25,clear sky,-13.8483,136.4192,
21,Rikitea,PF,78.39,scattered clouds,-23.1203,-134.9692,
24,Vaini,TO,80.60,few clouds,-21.2000,-175.2000,
27,Saint-Philippe,RE,78.80,clear sky,-21.3585,55.7679,
30,Kapaa,US,80.60,broken clouds,22.0752,-159.3190,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = clean.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Camiri,BO,80.60,overcast clouds,-20.0500,-63.5167,J R Hotel
6,Saint Anthony,US,84.20,few clouds,45.0205,-93.2180,Holiday Inn Express Roseville-St. Paul
9,Puerto Ayora,EC,86.00,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Galesong,ID,75.20,few clouds,-5.3166,119.3661,Sapriadi Appy
15,Alyangula,AU,81.25,clear sky,-13.8483,136.4192,Groote Eylandt Lodge
...,...,...,...,...,...,...,...
680,Hervey Bay,AU,75.00,broken clouds,-25.2986,152.8535,Shelly Bay Resort
687,Dwarka,IN,78.85,clear sky,22.2394,68.9678,Hotel Narayan Avezika Comfort
688,Bundaberg,AU,75.20,scattered clouds,-24.8500,152.3500,Takalvan Motel
694,Srivardhan,IN,77.83,clear sky,18.0333,73.0167,Savli Resort/Hotel - Shrivardhan


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd><dd> and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))